Training & Prediction part -> ok

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import numpy as np


# Path to your CSV file
file_path = 'data_simulation.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Dropping columns that are not useful for the prediction
df.drop(columns=['NOME', 'COGNOME', 'TELEFONO', 'CELLULARE', 'EMAIL', 'COMUNE', 'CAP', 'INDIRIZZO', 'CODICE_FISCALE', 'IBAN', 'COMUNE_NASCITA', 'NOME_AZIENDA', 'TIPO_AZIENDA', 'CODICE_FISCALE_AZIENDA', 'PARTITA_IVA_AZIENDA', 
                 'DOCUMENTAZIONE_PENSIONATO'], inplace=True)

# Define the number of quantiles
num_bins = 5
# Segment the TFR variable into quantiles
df['TFR_Category'] = pd.qcut(df['TFR'], q=num_bins, labels=[
                             'Very Low', 'Low', 'Medium', 'High', 'Very High'])
print(df['TFR'].dtypes)
print(df['TFR_Category'].dtypes)

# Define the number of quantiles
num_bins = 5
# Segment the TFR variable into quantiles
df['TFR_Category'] = pd.qcut(df['TFR'], q=num_bins, labels=[
                             'Very Low', 'Low', 'Medium', 'High', 'Very High'])
print(df['TFR'].dtypes)
print(df['TFR_Category'].dtypes)

df['TFR_Category'], bins = pd.qcut(df['TFR'], q=num_bins, labels=['Molto Basso', 'Basso', 'Medio', 'Alto', 'Molto Alto'], retbins=True)

for i in range(len(bins)-1):
    # print(f"{bins[i]} to {bins[i+1]} : {df['TFR_Category'].cat.categories[i]}")
    print(f"{df['TFR_Category'].cat.categories[i]} : {bins[i]} to {bins[i+1]}")
    

# Encode categorical variables
encoders = {}
for column in df.columns:
    if df[column].dtype == 'object' or df[column].dtype.name == 'category':
        le = LabelEncoder()
        df[column] = le.fit_transform(df[column])
        encoders[column] = le
print(encoders)


df.drop(columns=['TFR'], inplace=True)


# Define features and target variable
X = df.drop(columns=['MOTIVAZIONE_PRESTITO'])
y = df['MOTIVAZIONE_PRESTITO']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)


# Initialize and train the RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Get feature importances
importances = clf.feature_importances_

# Convert the importances into a DataFrame
feature_importances = pd.DataFrame(
    {'feature': X.columns, 'importance': importances})

# Sort the DataFrame by importance
feature_importances.sort_values(by='importance', ascending=False, inplace=True)

# Print the feature importances
print(feature_importances)

# Predict on the test set
y_pred = clf.predict(X_test)

# Evaluate the model
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred))

# Example new customer profile
new_customer = {
    'IMPORTO_RICHIESTO': 8000,
    'TIPO DI OCCUPAZIONE': 'Architetto comunale',
    'PROVINCIA': 'Genova',
    'CONSENSO_DATI_PRIVACY': 1,
    'CONSENSO_DATI_MRKTG': 1,
    'CONSENSO_DATI_CESSIONE_TERZI': 0,
    'SESSO': 'F',
    'REGIONE': 'Liguria',
    'COMUNE': 'City1',
    'CAP': '12345',
    'COMUNE_NASCITA': 'BirthCity1',
    'IMPORTO_STIPENDIO_PENSIONE': 2000,
    'TFR_Category': "Basso",
    'TIPO_AZIENDA': 'Pubblica',
    'TEMPO_INDETERMINATO': 1,
    'PREVENTIVI_CONCORRENZA': 0,
    'TRATTENUTE_BUSTA_PAGA_PENSIONE': 0,
    'ALTRI_FINANZIAMENTI_PRESENTI': 1
}

# Encode the new customer data using the fitted label encoders
print(encoders)

new_customer_encoded = {}

for column in new_customer:
    if column in encoders:
        new_customer_encoded[column] = encoders[column].transform(
            [new_customer[column]])[0]
    else:
        new_customer_encoded[column] = new_customer[column]

# Convert to DataFrame
new_customer_df = pd.DataFrame([new_customer_encoded])

# Ensure the new customer data has the same columns as the training data
new_customer_df = new_customer_df.reindex(columns=X.columns)

# Now you can make a prediction
predicted_motivation = clf.predict(new_customer_df)

predicted_motivation_label = encoders['MOTIVAZIONE_PRESTITO'].inverse_transform(
    predicted_motivation)

print(f"Predicted loan motivation for the new customer: {predicted_motivation_label[0]}")


# Predict the probabilities
predicted_probabilities = clf.predict_proba(new_customer_df)

# Convert the probabilities into a DataFrame
probabilities_df = pd.DataFrame(
    predicted_probabilities, columns=encoders['MOTIVAZIONE_PRESTITO'].classes_)

print(
    f"Predicted loan motivation for the new customer: {predicted_motivation_label[0]}")


print("Probabilities for each class:")
print(probabilities_df.transpose())

Training Part -> ok

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import numpy as np


# Define the function to categorize ages
def categorize_age(age):
    if 30 <= age <= 40:
        return "30-40"
    elif 41 <= age <= 50:
        return "41-50"
    elif 51 <= age <= 60:
        return "51-60"
    elif 61 <= age <= 70:
        return "61-70"
    else:
        return "Out of Range"

# Define the function to categorize working years


def categorize_wrk_yrs(wrkyrs):
    if 5 <= wrkyrs <= 10:
        return "5-10"
    elif 11 <= wrkyrs <= 20:
        return "11-20"
    elif 21 <= wrkyrs <= 30:
        return "21-30"
    elif 31 <= wrkyrs <= 40:
        return "31-40"
    elif 41 <= wrkyrs <= 45:
        return "41-45"
    else:
        return "Out of Range"


# Path to your CSV file
file_path = 'data_simulation.csv'
# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

df.columns

# ########### TFR to DROP ###################
# Define the number of quantiles
#####################################
num_bins = 5
df['TFR_Category'], bins = pd.qcut(df['TFR'], q=num_bins, labels=['Molto Basso', 'Basso', 'Medio', 'Alto', 'Molto Alto'], retbins=True)
#print(df['TFR'].dtypes)
#print(df['TFR_Category'].dtypes)

# Create a dictionary to store the range for each category
category_ranges = {
    'Molto Basso': (bins[0], bins[1]),
    'Basso': (bins[1], bins[2]),
    'Medio': (bins[2], bins[3]),
    'Alto': (bins[3], bins[4]),
    'Molto Alto': (bins[4], bins[5])
}

# Print the ranges for each category
print("\nTFR Categories and ranges")
for category, (low, high) in category_ranges.items():
    print(f'{category}: [{low} - {high}]')


# ########### AGE to DROP ###################
# Define the classe base on categorize_age
#####################################
# Apply the function to the DataFrame to create a new column
df['AGE_Category'] = df['AGE'].apply(categorize_age)


# ########### anni lavorativi to DROP ###################
# Define the classe base on categorize_age
#####################################
# Apply the function to the DataFrame to create a new column
df['anni_lavorativi_Category'] = df['anni lavorativi'].apply(categorize_wrk_yrs)

# besides, to drop are: TFR, AGE, anni lavorativi

# Dropping columns that are not useful for the prediction
df.drop(columns=['NOME',
                 'COGNOME',
                 'TELEFONO',
                 'CELLULARE',
                 'EMAIL',
                 'COMUNE',
                 'CAP',
                 'INDIRIZZO',
                 'CODICE_FISCALE',
                 'IBAN',
                 'COMUNE_NASCITA',
                 'AGE',
                 'anni lavorativi',
                 'TFR',
                 'NOME_AZIENDA',
                 'CODICE_FISCALE_AZIENDA',
                 'PARTITA_IVA_AZIENDA',
                 'DOCUMENTAZIONE_PENSIONATO',
                 'REGISTRAZIONE_TEL_PRIMO_CONTATTO',
                 'NOTE_LAVORAZIONE_CONTATTO'
                 ], inplace=True)

df.columns
#df.info()
#df['AGE_Category']


# Encode categorical variables
encoders = {}
for column in df.columns:
    if df[column].dtype == 'object' or df[column].dtype.name == 'category':
        le = LabelEncoder()
        df[column] = le.fit_transform(df[column])
        encoders[column] = le
#print(encoders)
#df.info()
#df['AGE_Category']


#Random Forest to train the model
# Define features and target variable
X = df.drop(columns=['MOTIVAZIONE_PRESTITO'])
y = df['MOTIVAZIONE_PRESTITO']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Initialize and train the RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Get feature importances
importances = clf.feature_importances_

# Convert the importances into a DataFrame
feature_importances = pd.DataFrame({'feature': X.columns, 'importance': importances})

# Sort the DataFrame by importance
feature_importances.sort_values(by='importance', ascending=False, inplace=True)

# Print the feature importances
print("\n")
print(feature_importances)

# Predict on the test set
y_pred = clf.predict(X_test)

# Evaluate the model
print(f"\nAccuracy: {accuracy_score(y_test, y_pred)}")
print("\n")
print(classification_report(y_test, y_pred))


TFR Categories and ranges
Molto Basso: [7254.55 - 44498.392]
Basso: [44498.392 - 72049.532]
Medio: [72049.532 - 107007.01200000003]
Alto: [107007.01200000003 - 160157.70800000004]
Molto Alto: [160157.70800000004 - 310022.36]


                           feature  importance
2                        PROVINCIA    0.574415
7                          REGIONE    0.164335
8                     DATA_NASCITA    0.037997
10  DATA_ ASSUNZIONE_PENSIONAMENTO    0.037807
1              TIPO DI OCCUPAZIONE    0.031464
0                IMPORTO_RICHIESTO    0.029168
16                    TFR_Category    0.015333
18        anni_lavorativi_Category    0.013752
17                    AGE_Category    0.012249
11                    TIPO_AZIENDA    0.012171
9       IMPORTO_STIPENDIO_PENSIONE    0.011896
13          PREVENTIVI_CONCORRENZA    0.007762
14  TRATTENUTE_BUSTA_PAGA_PENSIONE    0.007644
4              CONSENSO_DATI_MRKTG    0.007562
15   ALTRI_FINANZIAMENTI_ PRESENTI    0.007533
12             TEMPO

Prediction Part1 -> OK

In [5]:
# Example new customer profile
new_customer = {
    'IMPORTO_RICHIESTO': 15000,
    'TIPO DI OCCUPAZIONE': 'Architetto comunale',
    'PROVINCIA': 'Roma',
    'CONSENSO_DATI_PRIVACY': 1,
    'CONSENSO_DATI_MRKTG': 1,
    'CONSENSO_DATI_CESSIONE_TERZI': 0,
    'SESSO': 'F',
    'REGIONE': 'Lazio',
    'IMPORTO_STIPENDIO_PENSIONE': 2000,
    #'AGE_Category': '41-50',
    'anni_lavorativi_Category': '21-30',
    #'TFR_Category': "Basso",
    #'TIPO_AZIENDA': 'Pubblica',
    #'TEMPO_INDETERMINATO': 1,
    #'PREVENTIVI_CONCORRENZA': 0,
    #'TRATTENUTE_BUSTA_PAGA_PENSIONE': 0,
    'ALTRI_FINANZIAMENTI_PRESENTI': 1
}

# encoding new_customer data
new_customer_encoded = {}
for column in new_customer:
    if column in encoders:
        new_customer_encoded[column] = encoders[column].transform(
            [new_customer[column]])[0]
    else:
        new_customer_encoded[column] = new_customer[column]

#print(new_customer_encoded)


# Convert to DataFrame
new_customer_df = pd.DataFrame([new_customer_encoded])
# Ensure the new customer data has the same columns as the training data
new_customer_df = new_customer_df.reindex(columns=X.columns)
# Now you can make a prediction
predicted_motivation = clf.predict(new_customer_df)
predicted_motivation_label = encoders['MOTIVAZIONE_PRESTITO'].inverse_transform(predicted_motivation)

print(f"Predicted loan motivation for the new customer: {predicted_motivation_label[0]}")


# Predict the probabilities
predicted_probabilities = clf.predict_proba(new_customer_df)

# Convert the probabilities into a DataFrame
probabilities_df = pd.DataFrame(
    predicted_probabilities, columns=encoders['MOTIVAZIONE_PRESTITO'].classes_)

print(
    f"Predicted loan motivation for the new customer: {predicted_motivation_label[0]}")

print("Probabilities for each class:")
print(probabilities_df.transpose())

Predicted loan motivation for the new customer: spese viaggi
Predicted loan motivation for the new customer: spese viaggi
Probabilities for each class:
                             0
Corsi/Specializzazioni     0.0
Risarcimenti               0.0
acquisto arredamento casa  0.0
acquisto auto/moto         0.0
acquisto immobili          0.0
anticipo prima casa        0.0
consolidamento debiti      0.0
investimenti               0.0
liquidità                  0.0
non specificata            0.0
pagamenti imposte e tasse  0.0
rinegoziazione             0.0
ristrutturazione casa      0.0
spese dentistiche          0.0
spese matrimoniali         0.0
spese medico sanitarie     0.0
spese per cerimonie        0.0
spese universitarie        0.0
spese viaggi               1.0


Prediction Part2 -> flexible method

In [ ]:
# Example new customer profile
new_customer = {
    'IMPORTO_RICHIESTO': 15000,
    'TIPO DI OCCUPAZIONE': 'Architetto comunale',
    'PROVINCIA': 'Roma',
    'CONSENSO_DATI_PRIVACY': 1,
    'CONSENSO_DATI_MRKTG': 1,
    'CONSENSO_DATI_CESSIONE_TERZI': 0,
    # 'SESSO': 'F',
    'REGIONE': 'Lazio',
    # 'IMPORTO_STIPENDIO_PENSIONE': 2000,
    # 'AGE_Category': '41-50',
    'anni_lavorativi_Category': '21-30',
    # 'TFR_Category': "Basso",
    # 'TIPO_AZIENDA': 'Pubblica',
    # 'TEMPO_INDETERMINATO': 1,
    # 'PREVENTIVI_CONCORRENZA': 0,
    # 'TRATTENUTE_BUSTA_PAGA_PENSIONE': 0,
    'ALTRI_FINANZIAMENTI_PRESENTI': 1
}



# encoding new_customer data
new_customer_encoded = {}
for column in new_customer:
    if column in encoders:
        new_customer_encoded[column] = encoders[column].transform(
            [new_customer[column]])[0]
    else:
        new_customer_encoded[column] = new_customer[column]

print(new_customer_encoded)




In [ ]:


new_customer_encoded = {}
# Fill in missing features with median or mode values
for column in X.columns:
    if column not in new_customer:
        # If the column is numeric, fill with median value
        if X[column].dtype in ['int64', 'float64']:
            new_customer[column] = X[column].median()
        # If the column is categorical, fill with mode value
        else:
            new_customer[column] = X[column].mode()[0]

print(new_customer)

In [ ]:

# New customer profile with only three features
new_customer = {
    'IMPORTO_RICHIESTO': 8000,
    'TIPO DI OCCUPAZIONE': 'Poliziotto',
    'PROVINCIA': 'Roma'
}

new_customer_encoded = {}
# Fill in missing features with median or mode values
for column in X.columns:
    if column not in new_customer:
        # If the column is numeric, fill with median value
        if X[column].dtype in ['int64', 'float64']:
            new_customer[column] = X[column].median()
        # If the column is categorical, fill with mode value
        else:
            new_customer[column] = X[column].mode()[0]

print(new_customer)

In [ ]:
print(new_customer)
print(new_customer_encoded)

In [ ]:
from sklearn import tree
import matplotlib.pyplot as plt

# Select one of the trees in the forest
selected_tree = clf.estimators_[0]

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(4, 4), dpi=800)
tree.plot_tree(selected_tree,
               feature_names=X.columns,
               class_names=encoders['MOTIVAZIONE_PRESTITO'].classes_,
               filled=True)
fig.savefig('rf_individualtree.png')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import warnings
import os
os.environ['OMP_NUM_THREADS'] = '1'

warnings.filterwarnings('ignore', category=UserWarning)


# Display the columns of the DataFrame
print("Columns in the DataFrame:", df.columns)

# List of columns to be one-hot encoded
categorical_columns = ['TIPO DI OCCUPAZIONE', 'PROVINCIA', 'SESSO','REGIONE', 'COMUNE', 'CAP', 'COMUNE_NASCITA', 'TIPO_AZIENDA']

# Check if all the required columns are present in the DataFrame
missing_columns = [col for col in categorical_columns if col not in df.columns]
if missing_columns:
    print(
        f"The following columns are missing from the DataFrame and cannot be encoded: {missing_columns}")
else:
    # One-hot encode the categorical variables
    df_encoded = pd.get_dummies(df, columns=categorical_columns)

    # Standardize the features to have mean=0 and variance=1
    features_standardized = StandardScaler().fit_transform(df_encoded)

    # Perform PCA
    pca = PCA(n_components=2)
    principalComponents = pca.fit_transform(features_standardized)
    principalDf = pd.DataFrame(data=principalComponents, columns=[
                               'principal component 1', 'principal component 2'])

    # Perform KMeans clustering
    kmeans = KMeans(n_clusters=3)
    clusters = kmeans.fit_predict(features_standardized)

    # Add the cluster number to the principalDf
    principalDf['Cluster'] = clusters

    # Visualize the clusters
    plt.figure(figsize=(8, 6))
    sns.scatterplot(x="principal component 1", y="principal component 2",
                    hue="Cluster", data=principalDf, palette=['red', 'blue', 'green'])
    plt.show()

In [ ]:
import os
os.environ['OMP_NUM_THREADS'] = '1'

import warnings
warnings.filterwarnings('ignore', category=UserWarning)

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns

# One-hot encode the categorical variables
df_encoded = pd.get_dummies(df, columns=['TIPO DI OCCUPAZIONE', 'PROVINCIA','SESSO', 'REGIONE', 'COMUNE', 'CAP', 'COMUNE_NASCITA', 'TIPO_AZIENDA'])

# Standardize the features to have mean=0 and variance=1
features_standardized = StandardScaler().fit_transform(df_encoded)

# Perform PCA
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(features_standardized)
principalDf = pd.DataFrame(data=principalComponents, columns=[
                           'principal component 1', 'principal component 2'])

# Perform KMeans clustering
kmeans = KMeans(n_clusters=3)
clusters = kmeans.fit_predict(features_standardized)

# Add the cluster number to the principalDf
principalDf['Cluster'] = clusters

# Visualize the clusters
plt.figure(figsize=(8, 6))
sns.scatterplot(x="principal component 1", y="principal component 2", hue="Cluster", data=principalDf, palette=['red', 'blue', 'green'])
plt.show()

In [ ]:




# Calculate the sum of squared distances for different numbers of clusters
sum_of_squared_distances = []
K = range(1, 15)
for k in K:
    kmeans = KMeans(n_clusters=k)
    kmeans = kmeans.fit(features_standardized)
    sum_of_squared_distances.append(kmeans.inertia_)

# Plot the sum of squared distances
plt.figure(figsize=(8, 6))
plt.plot(K, sum_of_squared_distances, 'bx-')
plt.xlabel('k (number of clusters)')
plt.ylabel('Sum of Squared Distances')
plt.title('Elbow Method For Optimal k')
plt.show()

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Use PCA to reduce dimensionality so we can visualize the dataset on a 2D plot
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

# Convert to DataFrame
X_pca_df = pd.DataFrame(data=X_pca, columns=['PC1', 'PC2'])

# Concatenate the DataFrame with target labels
df_pca = pd.concat([X_pca_df, y], axis=1)

# Plot the clusters
plt.figure(figsize=(8, 6))
plt.scatter(df_pca.loc[df_pca['MOTIVAZIONE_PRESTITO'] == 0, 'PC1'],
            df_pca.loc[df_pca['MOTIVAZIONE_PRESTITO'] == 0, 'PC2'], color='blue', alpha=0.5, label='Car')
plt.scatter(df_pca.loc[df_pca['MOTIVAZIONE_PRESTITO'] == 1, 'PC1'],
            df_pca.loc[df_pca['MOTIVAZIONE_PRESTITO'] == 1, 'PC2'], color='red', alpha=0.5, label='House')
plt.scatter(df_pca.loc[df_pca['MOTIVAZIONE_PRESTITO'] == 2, 'PC1'],
            df_pca.loc[df_pca['MOTIVAZIONE_PRESTITO'] == 2, 'PC2'], color='green', alpha=0.5, label='Vacation')
plt.scatter(df_pca.loc[df_pca['MOTIVAZIONE_PRESTITO'] == 3, 'PC1'],
            df_pca.loc[df_pca['MOTIVAZIONE_PRESTITO'] == 3, 'PC2'], color='purple', alpha=0.5, label='Business')
plt.scatter(df_pca.loc[df_pca['MOTIVAZIONE_PRESTITO'] == 4, 'PC1'],
            df_pca.loc[df_pca['MOTIVAZIONE_PRESTITO'] == 4, 'PC2'], color='yellow', alpha=0.5, label='Education')
plt.legend()
plt.show()

In [ ]:
# Perform KMeans clustering
kmeans = KMeans(n_clusters=3)
clusters = kmeans.fit_predict(features_standardized)

# Convert the original data dictionary to a DataFrame
df_original = pd.DataFrame(data)

# Add the cluster assignments to the original DataFrame
df_original['Cluster'] = clusters

# Print the DataFrame with the cluster assignments
print(df_original)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import pairwise_distances
import matplotlib.pyplot as plt
import seaborn as sns

# Sample data
data = {
    'Feature1': np.random.rand(100),
    'Feature2': np.random.rand(100)
}
df = pd.DataFrame(data)

# Choose distance metric
metric = 'euclidean'  # Change this to 'manhattan', 'minkowski', etc. euclidean

# Perform Agglomerative Clustering
clustering = AgglomerativeClustering(
    n_clusters=3, metric=metric, linkage='ward')
labels = clustering.fit_predict(df)

# Add cluster labels to DataFrame
df['Cluster'] = labels

# Plot the clusters
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='Feature1', y='Feature2',
                hue='Cluster', palette='viridis')
plt.title('Agglomerative Clustering with {} distance'.format(metric))
plt.show()

In [ ]:
from sklearn.cluster import DBSCAN

# Choose distance metric
metric = 'euclidean'  # Change this to 'euclidean', 'minkowski', etc. manhattan

# Perform DBSCAN Clustering
dbscan = DBSCAN(eps=0.5, min_samples=5, metric=metric)
labels = dbscan.fit_predict(df)

# Add cluster labels to DataFrame
df['Cluster'] = labels

# Plot the clusters
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='Feature1', y='Feature2',
                hue='Cluster', palette='viridis')
plt.title('DBSCAN Clustering with {} distance'.format(metric))
plt.show()